In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pandas as pd
from dotenv import load_dotenv, find_dotenv
from sqlalchemy import create_engine, MetaData, Table

In [3]:
# подгружаем .env
load_dotenv()

True

In [4]:
# Считываем все креды
dst_host = os.environ.get('DB_DESTINATION_HOST')
dst_port = os.environ.get('DB_DESTINATION_PORT')
dst_username = os.environ.get('DB_DESTINATION_USER')
dst_password = os.environ.get('DB_DESTINATION_PASSWORD')
dst_db = os.environ.get('DB_DESTINATION_NAME')

s3_bucket = os.environ.get('S3_BUCKET_NAME')
s3_access_key = os.environ.get('AWS_ACCESS_KEY_ID')
s3_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY')

In [5]:
# Создадим соединения
dst_conn = create_engine(f'postgresql://{dst_username}:{dst_password}@{dst_host}:{dst_port}/{dst_db}')

In [6]:
# Укажем таблицу для удаления
TABLE_NAME = 'clean_real_estate_churn'

In [7]:
# Функция удаления таблицы с помощью sqlalchemy
def delete_table(table_name, conn):
    metadata = MetaData(bind=conn)
    table = Table(table_name, metadata, autoload=True)
    table.drop(conn)

In [8]:
# Удалим таблицу
delete_table(TABLE_NAME, dst_conn)

/tmp/ipykernel_103818/291150646.py:3: RemovedIn20Warning: Deprecated API features detected! These feature(s) are not compatible with SQLAlchemy 2.0. To prevent incompatible upgrades prior to updating applications, ensure requirements files are pinned to "sqlalchemy<2.0". Set environment variable SQLALCHEMY_WARN_20=1 to show all deprecation warnings.  Set environment variable SQLALCHEMY_SILENCE_UBER_WARNING=1 to silence this message. (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  metadata = MetaData(bind=conn)


In [9]:
# Проверим, что таблицы больше нет
pd.read_sql(f'select * from {TABLE_NAME}', dst_conn)

ProgrammingError: (psycopg2.errors.UndefinedTable) relation "clean_real_estate_churn" does not exist
LINE 1: select * from clean_real_estate_churn
                      ^

[SQL: select * from clean_real_estate_churn]
(Background on this error at: https://sqlalche.me/e/14/f405)

In [10]:
# Закроем соединения в конце работы
dst_conn.dispose()